## Connecting to elasticsearch

You can use the snippet below to connect to elasticsearch.

```python
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts=[ 'https://cast-elastic.bon-bon.org' ])
es.info()
```

## Searching for products

You can search for products using the `elasticsearch_dsl` module.  Here is a sample:

```python
from elasticsearch_dsl import *
base = Search(index='bigcommerce').using(es)
query = base.query('match_phrase_prefix', bin_picking_number='square')
results = query.execute(True)
```

The search above will search against the `bigcommerce` elasticsearch index.  If you recall,
the `bigcommerce` index contains a fairly up to date copy of the product information 
about products in elasticsearch.  The search above looks for all products whose 
`bin_picking_number` starts with `square`.  

## using your results

You loop through your results in order to see what you have:

```python
for x in results.hits[:10]:
    print(f'{x.name} {x.sku} {x.bin_picking_number}')
```

Here, for demonstration we use `[:10]` to limit our output to just the
first ten products.  Elasticsearch does not guarantee any sorting of the
results it returns to you.

## scanning all results

By default, elasticsearch will only return a small subset of all products to you.  
If you want all of the results, use `scan()`.  e.g.,

```python
for x in query.scan():
    print(f'{x.bin_picking_number}')
```

# children

Recall that products can have variants, such as different sizs or colors of a
particular product.  In elasticsearch, these are found in each products
`variants` attribute.  Here is an example of searching for all child
products that are associated with rgg:

```python
base = Search(index='bigcommerce').using(es)
query = base.query('prefix', variants__bin_picking_number__keyword='***rgg***')
results = query.execute(True)
for x in results.hits[:10]:
    for variant in x.variants:
        print(f'{variant.sku:30} {variant.bin_picking_number}')
```

We searched the variant-level `bin_picking_number` by prefixing it with `variants__`.
Note that when you use a `prefix` query, you have to use the `keyword` field like 
we did in the example above.

## creating a csv

Let's say you've gotten a query, and you want to format your results into a csv.
You can do that using the `csv` module which will format csvs for you automatically.
Here's an example of writing query results to a csv.

```python
base = Search(index='bigcommerce').using(es)
conditions = Q('prefix', bin_picking_number__keyword='***rgg***')
conditions |= Q('prefix', variants__bin_picking_number__keyword='***rgg***')
search = base.query(conditions)

import csv
with open('c:/Users/username/Documents/hello.txt', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow([ 'bigcommerce_product_id', 'bin_picking_number', 
                      'bigcommerce_variant_id', 'variant_bin_picking_number', 
                      'inventory_level' ])
    for product in search.scan():
        row = [ f'{product.id}', product.bin_picking_number, '', '', f'{product.inventory_level}' ]
        writer.writerow(row)
        if not product.variants:
            continue
        for variant in product.variants:
            row = [ f'{product.id}', product.bin_picking_number, 
                   f'{variant.id}', variant.bin_picking_number, f'{variant.inventory_level}' ]
            writer.writerow(row)
```

If you're paying attention, you'll see we've dropped you into some advanced query usage
above.  We've searched for products whose `bin_picking_number` is prefixed with `***rgg***` 
**OR** for products whose children have `bin_picking_number` prefixed with `***rgg***`.